## Scrapper de Leilões de carros
### Coisas importantes:
1. Modelo do carro
2. Ano do carro
3. Sinistro
4. Quilometragem
5. Data do leilão
6. Link para o carro

### Segunda etapa
1. Pelo link pegar o preco do maior lance
2. incremento min

Entra no link, pega todas informações de cada carro
passa pra proxima página, até a ultima
se achar algum que bate com o nome colocado de entrada
ele separa as informações e envia via WhatsApp

In [28]:
#-- IMPORT LIBS
from re import search
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.core.utils import ChromeType
from datetime import datetime
from pywhatkit import sendwhatmsg
import csv

def get_time() -> tuple:
    """
    Funcao para pegar a hora
    do local em que voce esta
    :return:
    """
    currentDateAndTime = datetime.now()

    currentTime_hours = currentDateAndTime.strftime("%H")
    currentTime_minutes = currentDateAndTime.strftime("%M")

    return currentTime_hours, currentTime_minutes

def navegador(start_url:str) -> any:
    """
    Funcao para iniciar o navegador chrome
    :param start_url:
    :return:
    """
    #-- OPTIONS TO START BROWSER
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--ignore-certificate-errors')

    #-- START BROWSER
    print('Abrindo o navegador...')
    chromium = ChromeDriverManager(chrome_type = ChromeType.CHROMIUM)
    driver = webdriver.Chrome(service = ChromiumService((chromium).install()), options=options)
    driver.get(start_url)

    return driver

def wpp(celular:str, msg:str) -> None:
    """
    Funcao para enviar msg no wpp,
    Quando for colocar o numero precisa do codigo
    do pais +55
    :param celular, msg:
    :return:
    """
    print('Em alguns segundos WhatsApp vai abrir e em 15 segundos enviar sua msg!')
    hora, minuto = get_time()
    sendwhatmsg(celular, msg, int(hora), int(minuto)+2, tab_close=True)

    return

def create_log() -> None:
    """
    Funcao para criar arquivo de logs
    :return:
    """
    with open('/logs.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames = ["modelo", "versao", "data_leilao",
                                                  "hora_leilao", "preco_inicial", "ano_carro",
                                                  "lance_atual", "link"])
        writer.writeheader()

    return


def log(dictionary:dict) -> None:
    """
    Funcao para adicionar arquivo de logs,
    para nao repetir carros ja enviados
    :param dictionary:
    :return:
    """
    with open('/logs.csv', 'a') as f:
        writer = csv.DictWriter(f)
        writer.writerow(dictionary)

    return

def get_num_pages():
    """
    Funcao para pegar o numero total de paginas
    :return:
    """
    driver = navegador('https://www.freitasleiloeiro.com.br/leiloes/pesquisar?pg=100&categoria=1')

    num_page = driver.find_element(By.CLASS_NAME, 'pagination').text
    numero_paginas = int(num_page[-2:])
    return numero_paginas

def scrapper(modelo_carro:str) -> list:
    """
    Scrapper para comecar a verificar os sites,
    Use o modelo do carro em letras maiusculas
    :param modelo_carro:
    :return:
    """

    for pag in range(get_num_pages()):

        driver = navegador('https://www.freitasleiloeiro.com.br/leiloes/pesquisar?pg=1&categoria=1')

        # Procura as descricoes dos carros
        carros = driver.find_elements(By.CLASS_NAME, 'cursor-pointer ')

        # Patterns para o regex search
        nome_carro = r'ONIX'
        pattern_model = r'(?=ONIX).+?(?<=,)'
        pattern_link_model = r"(?=').+(?<=')"
        pattern_data = r"[0-9]{2}\/[0-9]{2}\/[0-9]{4}"
        pattern_hora_leilao = r'[0-9]{2}:[0-9]{2}'
        patterno_preco_inicial = r'(?<=R\$ ).+'
        pattern_ano_carro = r"(?<=,).+?(?<=,)"
        pattern_lance_atual = r"R\$\s*([\d\.,]+)"
        aspas = r"'"
        vazio = ""

        cars_found = list()
        for i in carros:
            if search(nome_carro, i.text):
                # Separa as info do carro que foi encontrado
                link          = search(pattern_link_model, i.get_attribute("onclick")).group(0)
                link_carro    = f"https://www.freitasleiloeiro.com.br{link.replace(aspas, vazio)}"
                modelo        = search(pattern_model, i.text).group(0)
                data_leilao   = search(pattern_data, i.text).group(0)
                hora_leilao   = search(pattern_hora_leilao, i.text).group(0)
                preco_inicial = search(patterno_preco_inicial, i.text).group(0)
                ano_carro     = search(pattern_ano_carro, i.text).group(0)

                # Entra no link do carro que achou e pega o lance atual
                driver.get(link_carro)
                str_lance_atual = driver.find_element(By.ID, 'lance-atual').text
                lance_atual     = search(pattern_lance_atual, str_lance_atual).group(0)

                # Driver volta para pagina anterior para continuar
                driver.navigate().back()

                car =({
                    "nome_carro": nome_carro,
                    "modelo": modelo,
                    "data_leilao": data_leilao,
                    "hora_leilao": hora_leilao,
                    "preco_inicial": preco_inicial,
                    "ano_carro": ano_carro,
                    "lance_atual": lance_atual,
                    "link": link_carro})

                # Insere o log do carro que achou
                log(cars_found)
                # Adiciona o carro pra lista de carros encontrados
                cars_found.append(car)

        driver.quit()

    return cars_found

def filtra_carros_encontrados(cars_found:list) -> list:
    """
    Funcao para verificar se os carros encontrados pelo scrapper
    ja estao no log, se nao estiverem, envia msg
    :param cars_found:
    :return:
    """
    with open('/logs.csv', 'r') as f:
        log = csv.DictReader(f)

        # Cria uma lista com os links que o scrapper pegou
        link_values = [d['link'] for d in cars_found if 'link' in d]

        # Loop para tirar os carros que ja foram enviados, olhando o link
        for link in link_values:
            for row in log:
                if row['link'] == link:
                    del cars_found[link]
                    break
                if not link_already_exist:
                    return

    return cars_found

def envia_msg(list_dict_carros:list) -> None:
    """
    Funcao para enviar a msg filtrada no wpp
    :param list_dict_carros: lista de carros filtrados
    :return:
    """
    for carro in list_dict_carros:

        mensagem = f"Oportunidade!\n{carro.get(modelo)} {carro.get(ano_carro)}\nPreco inicial: {carro.get(preco_inicial)}\n{carro.get(lance_atual)}\nLeilao no dia: {carro.get(data_leilao)} as {carro.get(hora_leilao)}\nConfira o carro no link: {carro.get(link_carro)}"

        wpp("+55", mensagem)
    return

### Alguns testes que estou realizando enquanto faco o codigo principal estarao abaixo

In [30]:
driver = navegador('https://www.freitasleiloeiro.com.br/leiloes/pesquisar?pg=1&categoria=1')

carros = driver.find_elements(By.CLASS_NAME, 'cursor-pointer ')

nome_carro = r'ONIX'
pattern_model = r'(?=ONIX).+?(?<=,)'
pattern_link_model = r"(?=').+(?<=')"
pattern_data = r"[0-9]{2}\/[0-9]{2}\/[0-9]{4}"
pattern_hora_leilao = r'[0-9]{2}:[0-9]{2}'
patterno_preco_inicial = r'(?<=R\$ ).+'
pattern_ano_carro = r"(?<=,).+?(?<=,)"
aspas = r"'"
vazio = ""


for i in carros:
    if search(nome_carro, i.text):
        link = search(pattern_link_model, i.get_attribute("onclick")).group(0)
        link_carro = f"https://www.freitasleiloeiro.com.br{link.replace(aspas, vazio)}"
        modelo = search(pattern_model, i.text).group(0)
        data_leilao = search(pattern_data, i.text).group(0)
        hora_leilao = search(pattern_hora_leilao, i.text).group(0)
        preco_inicial = search(patterno_preco_inicial, i.text).group(0)
        ano_carro = search(pattern_ano_carro, i.text).group(0)

        driver.get(link_carro)
        lance_atual = driver.find_element(By.ID, 'lance-atual').text
        driver.close()

        # with open('/', 'a')

        mensagem = f"Oportunidade!\n{modelo} {ano_carro}\nPreco inicial: {preco_inicial}\n{lance_atual}\nLeilao no dia: {data_leilao} as {hora_leilao}\nConfira o carro no link: {link_carro}"

        wpp("+55", mensagem)

        break

Abrindo o navegador...
Em alguns segundos WhatsApp vai abrir e em 15 segundos enviar sua msg!
In 58 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
